# Read CMIP-PPE data and emulate with Gaussian Processor


**Adjusted for the eScience course from Duncan Watson-Parris' example here: [gist.github.com/duncanwp](https://gist.github.com/duncanwp/89175a17b7221e4d3639765621c7f7f9)**

You have to use the env:ml-notebook to run this example.  

In [31]:
import xarray as xr
import numpy as np
import pandas as pd
from esem import gp_model
from esem.utils import validation_plot, get_param_mask
from pathlib import Path
xr.set_options(display_style='html')
import intake
import cftime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import datetime
import seaborn as sns

In [32]:
def global_mean(ds):
    weights = np.cos(np.deg2rad(ds.lat))
    return ds.weighted(weights).mean(['lat', 'lon'])

def get_ensemble_member(ds):
    fname = ds.encoding['source']
    member = int(fname.split('.')[-4])
    return ds.assign_coords(member=member).expand_dims('member')

## Open the overview over the parameters in the CAM6 CESM PPE

In [33]:
data_path = Path('~/shared-craas1-ns9989k-ns9560k/CAM6_CESM_PPE/')

params = (xr.open_dataset(data_path / "parameter_262_w_control.nc")
          .to_pandas()
          .drop(columns = ['Sample_nmb'])
         )


### Open CMIP6 online catalog

In [34]:
cat_url = '/mnt/craas1-ns9989k-geo4992/data/catalogs/cesm-ppe.json'
col = intake.open_esm_datastore(cat_url)
col


### Search corresponding data 

Please check [here](https://pangeo-data.github.io/escience-2022/pangeo101/data_discovery.html?highlight=cmip6) for info about CMIP and variables :) 

Particularly useful is maybe the variable search which you find here: https://clipc-services.ceda.ac.uk/dreq/mipVars.html 

In [35]:
cat = col.search(
    experiment=['present-day'], 
    variable = ['SWCF'], 
    frequency='monthly'
)

cat.df


experiment  ensemble frequency variable units                long_name  \
0    present-day       0.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
1    present-day       1.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
2    present-day       2.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
3    present-day       3.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
4    present-day       4.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
..           ...       ...       ...      ...   ...                      ...   
257  present-day     258.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
258  present-day     259.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
259  present-day     260.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
260  present-day     261.0   monthly     SWCF  W/m2  Shortwave cloud forcing   
261  present-day     262.0   monthly     SWCF  W/m2  Shortwave cloud forcing   

     vertical_levels  start_time    end_time             time_range  \
0                1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
1                1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
2                1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
3                1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
4                1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
..               ...         ...         ...                    ...   
257              1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
258              1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
259              1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
260              1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   
261              1.0  0001-01-16  0003-12-16  0001-01-16-0003-12-16   

                                                  path  
0    /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
1    /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
2    /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
3    /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
4    /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
..                                                 ...  
257  /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
258  /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
259  /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
260  /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  
261  /mnt/craas1-ns9989k-ns9560k/CAM6_CESM_PPE/PD/m...  

[262 rows x 11 columns]

In [36]:
cat.df['variable'].unique()

array(['SWCF'], dtype=object)

### Create dictionary from the list of datasets we found
- This step may take several minutes so be patient!

In [37]:
dset_dict = cat.to_dataset_dict()#preprocess = get_ensemble_member,)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'experiment.frequency'


In [38]:
dset_dict['present-day.monthly']

<xarray.Dataset> Size: 2GB
Dimensions:   (lat: 192, lon: 288, time: 36, ensemble: 262)
Coordinates:
  * lat       (lat) float64 2kB -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon       (lon) float64 2kB 0.0 1.25 2.5 3.75 ... 355.0 356.2 357.5 358.8
  * time      (time) object 288B 0001-01-16 12:00:00 ... 0003-12-16 12:00:00
  * ensemble  (ensemble) float64 2kB 0.0 1.0 2.0 3.0 ... 259.0 260.0 261.0 262.0
Data variables:
    SWCF      (ensemble, time, lat, lon) float32 2GB dask.array<chunksize=(1, 36, 192, 288), meta=np.ndarray>
Attributes:
    intake_esm_vars:                   ['SWCF']
    intake_esm_attrs:experiment:       present-day
    intake_esm_attrs:frequency:        monthly
    intake_esm_attrs:variable:         SWCF
    intake_esm_attrs:units:            W/m2
    intake_esm_attrs:long_name:        Shortwave cloud forcing
    intake_esm_attrs:vertical_levels:  1.0
    intake_esm_attrs:start_time:       0001-01-16
    intake_esm_attrs:end_time:         0003-12-16
    intake_esm_attrs:time_range:       0001-01-16-0003-12-16
    intake_esm_attrs:_data_format_:    netcdf
    intake_esm_dataset_key:            present-day.monthly

In [39]:
ds = dset_dict['present-day.monthly']
SWCF = global_mean(ds['SWCF']).mean('time').compute()

In [40]:
# Some of the PPE ensemble members are missing data so just select the params we actually have
sub_params = params.iloc[SWCF.ensemble.values]
# Unit normalise all the parameters
ppe_params = (sub_params - sub_params.min()) / (sub_params.max() - sub_params.min())

In [41]:
# We can use an information criterion to choose the best parameters automatically:
best_params = ppe_params[ppe_params.columns[get_param_mask(ppe_params, SWCF)]]
best_params.columns

/opt/conda/envs/ml-notebook/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:688: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 9 iterations, i.e. alpha=4.397e-01, with an active set of 9 regressors, and the smallest cholesky pivot element being 5.960e-08. Reduce max_iter or increase eps parameters.
  warnings.warn(
/opt/conda/envs/ml-notebook/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:688: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 20 iterations, i.e. alpha=1.975e-01, with an active set of 20 regressors, and the smallest cholesky pivot element being 5.960e-08. Reduce max_iter or increase eps parameters.
  warnings.warn(
/opt/conda/envs/ml-notebook/lib/python3.11/site-packages/sklearn/linear_model/_least_angle.py:688: ConvergenceWarning: Regressors in active set degenerate. Dropping a regressor, after 22 iterations, i.e. alpha=1.077e-01, with an active set of 22 regr

Index(['micro_mg_autocon_nd_exp', 'micro_mg_dcs', 'cldfrc_dp1', 'cldfrc_dp2',
       'clubb_C6thlb', 'clubb_C8', 'clubb_c1', 'clubb_c11', 'clubb_c14',
       'dust_emis_fact', 'micro_mg_accre_enhan_fact', 'micro_mg_autocon_fact',
       'micro_mg_autocon_lwp_exp', 'micro_mg_berg_eff_factor',
       'micro_mg_homog_size', 'micro_mg_iaccr_factor', 'micro_mg_vtrmi_factor',
       'microp_aero_wsub_scale', 'microp_aero_wsubi_scale',
       'seasalt_emis_scale', 'zmconv_capelmt', 'zmconv_ke',
       'zmconv_tiedke_add'],
      dtype='object')

In [42]:
n_test = 25

X_test, X_train = best_params[:n_test], best_params[n_test:]
Y_test, Y_train = SWCF[:n_test], SWCF[n_test:]

## Global mean GP Model


In [43]:
# Can try different kernels here
gp = gp_model(X_train, Y_train, kernel=['Linear', 'RBF'])

In [44]:
gp.train()


In [45]:
m, v = gp.predict(X_test)


In [46]:
validation_plot(Y_test.data, m, v, figsize=(4,4))


Proportion of 'Bad' estimates : 8.00%


## Calibrate

In [47]:
from esem.utils import get_random_params
from esem.abc_sampler import ABCSampler, constrain

In [48]:
X_train

micro_mg_autocon_nd_exp  micro_mg_dcs  cldfrc_dp1  cldfrc_dp2  \
nmb_sim                                                                  
25                      0.971500      0.805847    0.206424    0.671471   
26                      0.649866      0.894649    0.618148    0.938361   
27                      0.703768      0.767592    0.051199    0.348967   
28                      0.948312      0.000000    0.888282    0.481579   
29                      0.123560      0.365255    0.674939    0.979595   
...                          ...           ...         ...         ...   
258                     0.725604      0.924849    0.092477    0.631673   
259                     0.633606      0.341112    0.076788    0.264890   
260                     0.619860      0.568149    0.765996    0.915039   
261                     0.992654      0.126066    0.255441    0.698665   
262                     0.297873      0.868720    0.830021    0.961661   

         clubb_C6thlb  clubb_C8  clubb_c1  clubb_c11  clubb_c14  \
nmb_sim                                                           
25           0.684229  0.793950  0.779754   0.227661   0.300534   
26           0.518324  0.538249  0.665506   0.812990   0.567041   
27           0.086444  0.549257  0.273296   0.979376   0.715680   
28           0.058440  0.964224  0.379381   0.530253   0.069777   
29           0.072070  0.271905  0.706585   0.654203   0.923545   
...               ...       ...       ...        ...        ...   
258          0.705486  0.747839  0.605231   0.586143   0.377838   
259          0.977408  0.668325  0.017842   0.878602   0.976971   
260          0.338742  0.983273  0.243526   0.099371   0.314178   
261          0.249564  0.683599  0.718731   0.917675   0.627753   
262          0.997973  0.375101  0.032601   0.586763   0.050706   

         dust_emis_fact  ...  micro_mg_berg_eff_factor  micro_mg_homog_size  \
nmb_sim                  ...                                                  
25             0.974161  ...                  0.106340             0.556251   
26             0.865573  ...                  0.032305             0.301909   
27             0.107133  ...                  0.783043             0.644865   
28             0.730504  ...                  0.756368             0.749765   
29             0.517474  ...                  0.887158             0.438671   
...                 ...  ...                       ...                  ...   
258            0.898157  ...                  0.357568             0.320785   
259            0.009132  ...                  0.971503             0.063019   
260            0.179294  ...                  0.693411             0.448854   
261            0.204757  ...                  0.416335             0.667422   
262            0.727236  ...                  0.159003             0.856063   

         micro_mg_iaccr_factor  micro_mg_vtrmi_factor  microp_aero_wsub_scale  \
nmb_sim                                                                         
25                    0.469144               0.505458                0.931798   
26                    0.499515               0.088459                0.731500   
27                    0.811843               0.905836                0.991289   
28                    0.772850               0.655938                0.489364   
29                    0.960535               0.433522                0.682204   
...                        ...                    ...                     ...   
258                   0.139384               0.989630                0.379718   
259                   0.132357               0.034985                0.057183   
260                   0.205011               0.371056                0.273651   
261                   0.226351               0.178084                0.944702   
262                   0.925182               0.108261                0.105863   

         microp_aero_wsubi_scale  seasalt_emis_scale  zmconv_capelmt  \
nmb_sim             

In [49]:
# Setup sampler with 1 million points
sample_points = pd.DataFrame(data=get_random_params(23, int(1e6)), columns=X_train.columns)
sampler = ABCSampler(gp, np.asarray([-40.5]), obs_uncertainty=0.5)

In [50]:
valid_samples = sampler.batch_constrain(sample_points, batch_size=10000)


  0%|          | 0/1000000 [00:00<?, ?sample/s]

In [51]:
print("Remaining points: {}".format(valid_samples.sum()))


Remaining points: 100


In [52]:
type(valid_samples)

numpy.ndarray